# EJEMPLOS DE REDES PREENTRENADAS

## Paso1: Importar librerías 

In [1]:
import tensorflow as tf
import numpy as np

2024-12-19 19:50:35.372265: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Paso 2: Preparación de Datos
Fashion-MNIST es un conjunto de datos de imágenes de artículos de Zalando que consta de un conjunto de entrenamiento de 60000 ejemplos y un conjunto de prueba de 10000 ejemplos. Cada ejemplo es una imagen en escala de grises de 28x28, asociada con una etiqueta de 10 clases diferentes.


In [2]:
#  Construir una red neuronal convolucional para Fashion MNIST 
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

## Paso3: División de Datos

Vamos a separar fashion MNIST en dos conjuntos de entrenamiento:
* `X_train_A`: todas las imagenes excepto T-shirts/tops y pullovers (clases 0 y 2).
* `X_train_B`: un conjunto pequeño con las 200 primeras imagenes de T-shirts/tops y pullovers.

El conjunto de validación y el conjunto de pruebas también se dividen de esta manera, pero sin restringir la cantidad de imágenes.

In [3]:
#  Cargar los datos de entrenamiento y test de Fashion MNIST 
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
# Se recuperan los ultimos 5000 datos de entrenamiento para usarlos como datos de validacion
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
# Procedemos a normalizar los datos. Se divide por 255 para que los valores esten entre 0 y 1. Esto se hace con el fin de que el entrenamiento sea mas rapido.
# Para los datos de validacion y test no se hace esto, ya que no se deben modificar los datos de validacion y test.
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

In [4]:
# Definir los nombres de las clases del dataset
class_names = ["T-shirt/top","Trouser","Pullover", "Dress","Coat","Sandal","Shirt", "Sneaker","Bag", "Ankle boot",]

In [ ]:
# Se definen las dos clases que se van a usar para el problema de claseificacion binaria
pos_class_id = class_names.index("Pullover") #2
neg_class_id = class_names.index("T-shirt/top") #0

# La función split_dataset es una función diseñada para dividir un conjunto de datos en dos subconjuntos,
# A y B, donde el subconjunto A está destinado a una tarea de clasificación multiclase con 8 clases, 
# y el subconjunto B está destinado a una tarea de clasificación binaria.
def split_dataset(X, y):
    # y_for_B -> Crear una máscara booleana para identificar las clases positivas y negativas en y
    # y_A -> Crear subconjunto A excluyendo ejemplos de clases positivas y negativas
    # y_B -> Crear subconjunto B con clases positivas codificadas como 1 y otras clases como 0
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)

    # Reetiqueta clases en el subconjunto A para que vayan de 0 a 7
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id
    # Devuelve tupla con subconjunto A (X, y_A) y subconjunto B (X, y_B)
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

#  dividen los conjuntos de datos originales en subconjuntos A y B, y limitan el tamaño del subconjunto B de entrenamiento a 200 ejemplos
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

[1, 3, 4, 5, 6, 7, 8, 9]
[1, 3, 4, 5, 6, 7, 8, 9]
[1, 3, 4, 5, 6, 7, 8, 9]


## Paso 4 y 5: Creación y entrenamiento de los modelos

En este caso se van a crear dos modelos A y B. Se crea el primer modelo A, se entrena y se guarda. Una vez guardado, se crea el segundo modelo. Para la creación de este segundo modelo vamos a mostrar dos opciones. La primera creando un segundo modelo sin transferencia, para ver los resultados. La segunda opción será crear un segundo modelo B con transferencia de capas entrenadas del modelo A. De esta forma podremos comparar la eficiencia de utilizar modelos preentrenados para crear parte de nuestro modelo. 

##### Creamos el primer modelo de tipo secuencial - modelo A

In [15]:
#Limpiea de sesión y establece semilla aleatoria de tamaño 42
tf.keras.backend.clear_session()
tf.random.set_seed(42)

#Se define el modelo A que es el modelo de clasificación multiclase
# Se crea un modelo secuencial con 4 capas densas, la primera capa es la capa de entrada que recibe un tensor de 28x28 y lo aplana
# Las siguientes 3 capas son capas densas con 100 neuronas cada una, función de activación relu y kernel_initializer he_normal
model_A = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(28, 28)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(8, activation="softmax"),
    ]
)

# Se compila el modelo A con la función de pérdida sparse_categorical_crossentropy, el optimizador SGD con learning_rate=0.001 y la métrica accuracy
model_A.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
    metrics=["accuracy"],
)

In [17]:
history = model_A.fit(X_train_A, y_train_A, epochs=20, validation_data=(X_valid_A, y_valid_A))
model_A.save("Resultados/my_model_A.keras")

Epoch 1/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9128 - loss: 0.2526 - val_accuracy: 0.9030 - val_loss: 0.2686
Epoch 2/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9143 - loss: 0.2494 - val_accuracy: 0.9037 - val_loss: 0.2661
Epoch 3/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9154 - loss: 0.2464 - val_accuracy: 0.9042 - val_loss: 0.2638
Epoch 4/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9162 - loss: 0.2435 - val_accuracy: 0.9057 - val_loss: 0.2616
Epoch 5/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9171 - loss: 0.2408 - val_accuracy: 0.9065 - val_loss: 0.2595
Epoch 6/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9179 - loss: 0.2383 - val_accuracy: 0.9080 - val_loss: 0.2575
Epoch 7/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9190 - loss: 0.2358 - val_accuracy: 0.9090 - val_loss: 0.2556
Epoch 8/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9199 - loss: 0.2334 - 

##### Se crea el segundo modelo B de tipo secuencial. 2 opciones:

##### Opcion1: Modelo B sin transferencia
A continuación entrenamos y evaluamos el modelo B sin utilizar el modelo A para trasnferir los conocimientos previos del modelo. 

In [18]:
# Entrenando y evaluando el modelo B sin usar el modelo A
tf.random.set_seed(42)
model_B = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20, validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.2576 - loss: 0.8675 - val_accuracy: 0.3116 - val_loss: 0.7882
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3031 - loss: 0.8026 - val_accuracy: 0.3947 - val_loss: 0.7417
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4176 - loss: 0.7488 - val_accuracy: 0.5292 - val_loss: 0.7015
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5694 - loss: 0.7027 - val_accuracy: 0.6360 - val_loss: 0.6655
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6705 - loss: 0.6614 - val_accuracy: 0.7310 - val_loss: 0.6329
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7603 - loss: 0.6242 - val_accuracy: 0.7883 - val_loss: 0.6036
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8225 - loss: 0.5910 - val_accuracy: 0.8309 - val_loss: 0.5776
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8773 - loss: 0.5618 - val_accuracy: 0.8526 - val_loss: 0.5547
Epoc

Opcion 2_1: 
Vamos a reutilizar ahora el modelo A para transferir conocimiento al modelo B y entrenarlo. 
1. Carga del modelo guardado: carga el modelo preentrenado desde el archivo guardado. 
2. Transferencia de las capas a utilizar al segundo modelo: 
    + Crea un nuevo modelo llamado model_B_on_A utilizando la clase Sequential de Keras.
    + Se toman todas las capas del modelo model_A excepto la última capa (utilizando model_A.layers[:-1])
3. Se añade una ultima capa densa nueva con una neurona al modelo B: Esta capa tiene 1 neurona (ya que es una tarea de clasificación binaria) y utiliza la función de activación sigmoidal ("sigmoid"), que es comúnmente utilizada en la capa de salida para problemas de clasificación binaria.

In [19]:
model_A = tf.keras.models.load_model("Resultados/my_model_A.keras")
# Se transfieren todas las capas menos la última del modelo A al modelo B
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
#Se crea una nueva capa densa con 1 neurona y función de activación sigmoidal para la clasificación binaria
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

Opcion 2_2: 
Creamos de nuevo el modelo model_B_on_A pero esta vez clonando el modelo A ya que sino al entrenar el modelo B el modelo A también se ve afectado:
1. Clona el modelo A:
    + Crea una copia exacta (model_A_clone) del modelo model_A utilizando la función clone_model. Esto crea un nuevo modelo con la misma arquitectura que model_A, pero sin compartir los mismos pesos.
    + Establece los pesos de model_A_clone con los mismos pesos que tiene el modelo original model_A. Esto es esencial para hacer que model_A_clone tenga los mismos parámetros de peso que model_A, de modo que sea una copia exacta.
    + Se realiza la transferencia de pesos preentrenados

2. Se traslada al modelo model_B_on_A las capas clonadas y sus pesos (excepto la última).
3. Se añade una última capa densa nueva con una neurona al modelo B: esta capa tiene 1 neurona (ya que es una tarea de clasificación binaria)
4. Se congelan las capas preentrenadas: Es necesario iterar sobre todas las capas de model_B_on_A excepto las agregadas por nosotros y establecer la propiedad trainable de cada capa en False. Evita que los pesos de esas capas se actualicen durante las primeras etapas del ajuste fino.

La congelación y descongelación de capas es una estrategia para controlar cuándo y cómo se actualizan los pesos de las capas preentrenadas durante el proceso de entrenamiento. Esto puede ser beneficioso cuando se tiene un conjunto de datos limitado para la tarea específica.


In [21]:
tf.random.set_seed(42)
model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

# Se congela todas las capas del modelo A
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

In [22]:
# Se compila el modelo B sobre el modelo A con la función de pérdida binary_crossentropy, el optimizador SGD con learning_rate=0.001 y la métrica accuracy
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# Se entrena el modelo B sobre el modelo A con 4 épocas y los datos de validación B 
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.4350 - loss: 1.3379 - val_accuracy: 0.4797 - val_loss: 0.8819
Epoch 2/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3883 - loss: 0.9160 - val_accuracy: 0.4570 - val_loss: 0.7658
Epoch 3/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3524 - loss: 0.8123 - val_accuracy: 0.4570 - val_loss: 0.7314
Epoch 4/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3803 - loss: 0.7738 - val_accuracy: 0.4906 - val_loss: 0.7077


Descongelación de capas preentrenadas: se vuelve a cambiar la propiedad trainable a True para las capas preentrenadas. Esto significa que ahora las capas preentrenadas pueden aprender a partir de los datos de la tarea específica.

In [23]:
# Se ponen todas las capas del modelo B sobre el modelo A como entrenables, lo que permitirá recarcular los pesos
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

Compilación y entrenamiento final: Se compila y entrena el modelo completo (con todas las capas, incluidas las preentrenadas y las añadidas para la tarea específica) durante más épocas. Esto permite que el modelo ajuste finamente los pesos de todas las capas para la tarea específica.

In [24]:
# Se compila de nuevo el modelo B sobre el modelo A con la función de pérdida binary_crossentropy, el optimizador SGD con learning_rate=0.001 y la métrica accuracy
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# Se entrena el modelo B sobre el modelo A con 16 épocas y los datos de validación B
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.4888 - loss: 0.7325 - val_accuracy: 0.6271 - val_loss: 0.6335
Epoch 2/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6300 - loss: 0.6506 - val_accuracy: 0.7428 - val_loss: 0.5718
Epoch 3/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7122 - loss: 0.5826 - val_accuracy: 0.7953 - val_loss: 0.5230
Epoch 4/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8152 - loss: 0.5295 - val_accuracy: 0.8388 - val_loss: 0.4845
Epoch 5/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8560 - loss: 0.4871 - val_accuracy: 0.8694 - val_loss: 0.4532
Epoch 6/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8810 - loss: 0.4526 - val_accuracy: 0.8912 - val_loss: 0.4271
Epoch 7/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9053 - loss: 0.4239 - val_accuracy: 0.9001 - val_loss: 0.4049
Epoch 8/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9161 - loss: 0.3995 - val_accuracy: 0.9080 - val_loss: 0.3857
Epo

## Paso 6: Evaluación el modelo con los datos de testing

Evaluamos ambos modelos para comparar la diferencia de utilizar la transferencia de modelos preentrenados.

In [25]:
# Evaluando el modelo B  sobre el conjunto de test 
model_B.evaluate(X_test_B, y_test_B)

# Evaluando el modelo B sobre el modelo A  sobre el conjunto de test 
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.9016 - loss: 0.3988
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - accuracy: 0.9336 - loss: 0.2973


[0.3008052408695221, 0.9284999966621399]